## HQC

In [70]:
%run ProductCode.ipynb
%run ConcatenatedCode.ipynb
%run Conversions.ipynb

In [71]:
class HQC:
    
    def __init__(self, w, w_e, w_r, C, key_type = 'pol'):
        
        self.w = w
        self.w_e = w_e
        self.w_r = w_r
        self.C = C
        self.q = 2
        
        self.p, self.power = is_prime_power(self.q, get_data = True)
        
        self.n = self.C.n
        self.k = self.C.k
        self.delta = self.C.delta
        
        self.F = GF(self.q)
        self.R = PolynomialRing(self.F, 'x'); x = self.R.gen()
        self.S = QuotientRing(self.R, x**self.n - 1, 'a'); a = self.S.gen()
        
        self.pk, self.sk = self.KeyGen(out = key_type)
        
        
    def KeyGen(self, out = 'pol'):
        
        h = self.R.random_element(degree=(0,self.n))
        h = self.S(h)
        
        x = [0] * self.n
        x = self.RandomGenWithWeight(x, self.w)
        x = self.S(x)
        
        y = [0] * self.n
        y = self.RandomGenWithWeight(y, self.w)
        y = self.S(y)
        
        s = x + h * y
        
        if out == 'pol':
            pk = (h, s)
            sk = (x, y)
        elif out == 'int':
            pk = (_PolToInt(h, self.q), _PolToInt(s, self.q))
            sk = (_PolToInt(x, self.q), _PolToInt(y, self.q))
        elif out == 'bin':
            pk = (_PolToInt(h, self.q), _PolToInt(s, self.q))
            sk = (_PolToInt(x, self.q), _PolToInt(y, self.q))
            pk = (_IntToBitString(pk[0], self.q), _IntToBitString(pk[1], self.q))
            sk = (_IntToBitString(sk[0], self.q), _IntToBitString(sk[1], self.q))
        
        return pk, sk
    
    
    def Encrypt(self, message, zeropad = True, out = 'bin'):
        
        pk_data_type = _DetermineInput(self.pk[0], self.q)
        
        if pk_data_type == 'pol':
            pk = self.pk
        elif pk_data_type == 'int':
            pk = (self.S(self.pk[0]), self.S(self.pk[1]))
        elif pk_data_type == 'bin':
            pk = (_BitStringToInt(self.pk[0], self.q), _BitStringToInt(self.pk[1], self.q))
            pk = (self.S(pk[0]), self.S(pk[1]))
        
        
        c = self.C.Encoding(message, out = 'int')
        
        #print(c)
        #for i in range(len(c)):
        #    c[i] = self.F(c[i])
        #print(c)
        
        #return c
    
        c = _BitStringToInt(c, self.q)
        
        u = []
        v = []
        
        for i in range(0, len(c), self.n):
            e = [0] * self.n
            e = self.RandomGenWithWeight(e, self.w_e)
            e = self.S(e)

            r1 = [0] * self.n
            r1 = self.RandomGenWithWeight(r1, self.w_r)
            r1 = self.S(r1)

            r2 = [0] * self.n
            r2 = self.RandomGenWithWeight(r2, self.w_r)
            r2 = self.S(r2)
            
            u.extend(r1 + pk[0] * r2)
            v.extend(self.S(c[i:i+self.n]) + pk[1] * r2 + e)
            
        if out == 'int':
            u, v = (_PolToInt(u, self.q), _PolToInt(v, self.q))
        elif out == 'bin':
            u, v = _PolToInt(u, self.q), _PolToInt(v, self.q)
            u, v = (_IntToBitString(u, self.q), _IntToBitString(v, self.q))
        else:
            raise ValueError('Unrecognized data type')
            
        return (u, v)
            
    
    def Decrypt(self, c, out = 'bin'):
        
        sk_data_type = _DetermineInput(self.sk[0], self.q)
        
        if sk_data_type == 'pol':
            sk = self.sk
        elif sk_data_type == 'int':
            sk = (self.S(self.sk[0]), self.S(self.sk[1]))
        elif sk_data_type == 'bin':
            sk = (_BitStringToInt(self.sk[0], self.q), _BitStringToInt(self.sk[1], self.q))
            sk = (self.S(sk[0]), self.S(sk[1]))
            
            
        c_data_type = _DetermineInput(c[0], self.q)
        
        if c_data_type == 'int':
            pass
        elif c_data_type == 'bin':
            c = (_BitStringToInt(c[0], self.q), _BitStringToInt(c[1], self.q))
        
        
        d_out = []
        
        for i in range(0, len(c[0]), self.n):
            d = self.S(c[1][i:i+self.n]) - self.S(c[0][i:i+self.n]) * sk[1]
            d = _PolToInt(d, self.q)
            d = self.C.Decoding(d, out = 'bin')
            d_out.extend(_BitStringToInt(d, self.q))
        
        
        if out == 'int':
            pass
        elif out == 'bin':
            d_out = _IntToBitString(d_out, self.q)
        else:
            raise ValueError('Unrecognized data type')
        
        return d_out
        
        
    def RandomGenWithWeight(self, array, weight):

        for i in range(weight):
            cur = ZZ.random_element(0,len(array))
            while array[cur] == 1:
                cur = ZZ.random_element(0,len(array))
            
            array[cur] = 1
        
        return array

In [72]:
C = HQC(w = 3, w_e = 3, w_r = 3, C = ProductCode(BCHCode(n = 15, b = 1, D = 7), RepetitionCode(n = 31)), key_type = 'pol')
#C = HQC(w = 3, w_e = 3, w_r = 3, C = ProductCode(RSCode(n=9, k=5, q=2**4), RSCode(n=7, k=3, q=2**4)), key_type = 'pol')

#C = HQC(w = 2, w_e = 2, w_r = 2, C = ProductCode(RMCode(r = 1, m = 3), RMCode(r = 2, m = 6)), key_type = 'pol')



print('Public key: ', C.pk)
print('\nSecret key: ', C.sk)

Public key:  (a^393 + a^392 + a^389 + a^388 + a^386 + a^382 + a^381 + a^379 + a^376 + a^375 + a^373 + a^366 + a^364 + a^363 + a^360 + a^358 + a^355 + a^354 + a^353 + a^349 + a^347 + a^339 + a^338 + a^336 + a^335 + a^333 + a^332 + a^330 + a^328 + a^327 + a^326 + a^323 + a^322 + a^319 + a^318 + a^317 + a^314 + a^313 + a^312 + a^311 + a^309 + a^308 + a^304 + a^303 + a^301 + a^299 + a^296 + a^295 + a^294 + a^290 + a^288 + a^284 + a^282 + a^278 + a^277 + a^275 + a^272 + a^266 + a^259 + a^258 + a^257 + a^256 + a^255 + a^254 + a^251 + a^247 + a^246 + a^245 + a^243 + a^242 + a^239 + a^238 + a^236 + a^235 + a^234 + a^233 + a^231 + a^230 + a^223 + a^222 + a^219 + a^214 + a^212 + a^211 + a^210 + a^207 + a^206 + a^205 + a^204 + a^203 + a^201 + a^200 + a^199 + a^196 + a^194 + a^190 + a^189 + a^188 + a^186 + a^176 + a^175 + a^171 + a^169 + a^168 + a^167 + a^166 + a^165 + a^164 + a^163 + a^160 + a^159 + a^157 + a^156 + a^155 + a^154 + a^153 + a^152 + a^150 + a^149 + a^148 + a^144 + a^143 + a^140 + a^

In [73]:
C.k

5

In [74]:
m = '11101111010100011100'
#m = [1,1,1,0,1,1,1,1,0,1,0,1,0,0,0]
#m = vector(GF(2), [1,0,1,1,1,1,1])

c = C.Encrypt(m, out = 'bin')
print(c)


('01101110001111110001001001000010011000101000010111011001001101010111110001000101010110101001010110111011010001001111000000011101011110110000000000000011100100110101111100010000101001110000001011001101000101100000101100101011110110110001100010110000111000000010000111010001000111000101101111111110011100001010001111010100000000111001111101111001001000100111111100000001111101111110111100111010011111000110000001101010111010100000011010010000000010011010100001001011110101010111101000011100000110101001010001000110000011001111101010001011111001011111111000001101100000011000010011101111010100101110110101110101110100111000111000101001111111110010111000010110001010011111100110110111001111110010100001110011001011100101010110000101110100101101101110101011100001110100101011110000000110101101010101011000100100010011100011100101000110011011000011001001101110001111100011100110101100001010001110010000010001101100000110110001011101100011001110010101100011011110001110100101010010001011000000001110000100

In [75]:
d = C.Decrypt(c, out = 'bin')
print(d)

11101111010100011100


In [ ]:
15*63